# <p align="center"> Notebook dùng để chạy giải thuật JSSP version 2 </p>

In [35]:
import sys
if "../" not in sys.path:
    sys.path.append("../")
from validation import checkCycle
from mip_solver import LPJS
from greedy import GreedyJS

## Mở file Excel sau đây để xem chi tiết các công việc cần lập lịch

### Có thể thay đổi các thông số trong file này để thực hiện lập lịch cho các công việc khác tùy ý, và cần phải lưu lại file trước khi thực hiện các bước tiếp theo. Cách thêm hoặc thay đổi:
- Các công việc ở sheet `Jobs`, bao gồm:
    - `Job ID` : mã số của công việc (bắt buộc, không được trùng với các mã số khác)
    - `Job Name` : tên của công việc (không bắt buộc)
    - `Job Description` : mô tả công việc (không bắt buộc)
- Các tác vụ ở sheet `Tasks`, bao gồm:
    - `Task ID` : mã số của tác vụ (bắt buộc, không được trùng với các mã số khác)
    - `Task Name` : tên của tác vụ (không bắt buộc)
    - `Task Description` : mô tả tác vụ (không bắt buộc)
- Thứ tự thực hiện của tác vụ ở sheet `TaskOrders`, bao gồm:
    - `First Task ID` : mã số của tác vụ được thực hiện đầu trước
    - `Subsequent Task ID` : mã số của tác vụ được thực hiện sau
- Các máy ở sheet `Machines`, bao gồm:
    - `Machine ID` : mã số của máy (bắt buộc, không được trùng với các mã số khác)
    - `Machine Name` : tên của máy (không bắt buộc)  
- Các thông số của máy ở sheet `Capacities`, bao gồm:
    - `Machine ID` : mã số của máy
    - `Task ID` : mã số của tác vụ
    - `Time` : thời gian thực hiện tác vụ trên máy
    - `Cost` : chi phí thực hiện tác vụ trên máy

In [36]:
import ipywidgets as widgets
from IPython.display import display
import os
def open_data_file(_):
    file_path = r"..\datasets\Data.xlsx"
    if os.path.exists(file_path):
        os.startfile(file_path)
    else:
        print("File not found.")

w = widgets.Button(description='Open data file')
w.on_click(open_data_file)
display(w)


Button(description='Open data file', style=ButtonStyle())

## Sau khi đã thay đổi các thông số, chạy các cell bên dưới để tiến hành lập lịch

In [37]:
import pandas as pd
capacities = pd.read_excel('../datasets/Data.xlsx',sheet_name='Capacities')
from collections import defaultdict

cap = defaultdict(dict)
for index, row in capacities.iterrows():
    machineid = int(row["Machine ID"])
    cap[machineid][int(row["Task ID"])] = (row["Time"], row["Cost"])
capacities = cap


In [38]:
%%html
<style>
.mytext > .widget-label {
    color: #066889;
    font-size: 20px;
}
</style>

### 1. Chạy cell bên dưới để chọn số lượng công việc cần lập lịch, sau đó với mỗi công việc, nhập mã tác vụ cần thực hiện và thứ giữa các tác vụ đó. Ví dụ:
 - Số lượng công việc cần lập lịch: 2
    - Công việc 1: 
        - Các tác vụ: 1, 2, 3 (nhập ID tác vụ cách nhau bởi dấu phẩy)
        - Thứ tự thực hiện: (1,2), (2,3) (nhập thứ tự thực hiện cách nhau bởi dấu phẩy, mỗi cặp tác vụ cách nhau bởi dấu phẩy và được bao bởi dấu ngoặc đơn)
        - Số lần lặp: 1 (mặc định là 1, có thể thay đổi)
    - Công việc 2:
        - Các tác vụ: 4, 5, 6
        - Thứ tự thực hiện: (4,5)
        - Số lần lặp: 2

### 2. Chọn mô hình:
- MIP : mô hình tối ưu nguyên, mặc định. Các tham số có thể được điều chỉnh trong hàm `solve`
    - `display` : hiển thị Gannt chart, mặc định là `True`
    - `time_limit` : thời gian giới hạn lập lịch, mặc định là 10 giây.
- Greedy : mô hình tham lam, lựa chọn các máy có chi phí thực hiện thấp nhất để thực hiện tác vụ
    - display : hiển thị Gannt chart, mặc định là `True`

### 3. Nhấn nút **Solve** để lập lịch và hiển thị kết quả

### 4. Trong quá trình lập lịch, có thể thay đổi các thông số và nhấn nút **Solve** để lập lịch lại, tuy nhiên nên chạy lại cell để output được hiển thị đầy đủ, không bị đè lên output của lần lập lịch trước đó
---
**_Chú ý:_**  
- **_Các thông số về tác vụ không được để trống_**
- **_Không nhập các tác vụ không tồn tại trong file dữ liệu_**
- **_Không nhập các thứ tự thực hiện không chứa trong các tác vụ đã nhập_**
- **_Không nhập các thứ tự thực hiện không hợp lệ (ví dụ: (1,2) và (2,1) cùng tồn tại)_**
---
**_Nếu nhập sai, chương trình sẽ báo lỗi, khi đó chạy lại cell để nhập lại_**

In [39]:
import ipywidgets as widgets
from ipywidgets import Layout, Button, VBox, HBox, Label, Text

# Step 2
num_jobs = widgets.IntText(
    value=1,
    min=1,
    max=200,
    description= "Chọn số lượng công việc:",
    style={"description_width": "initial", "handle_color": "lightblue"},
    layout=Layout(width="25%"),
)
num_jobs.add_class("mytext")
output = widgets.Output()
tasks_input = []
orders_input = []
times = []  # number of times each job repeats


# Step 5
def on_button_click(b):
    output.clear_output()
    assert 51 > num_jobs.value > 0, "Số lượng công việc phải lớn hơn 0 và nhỏ hơn 50"
    global tasks_input, orders_input, times
    tasks_input = [
        Text(
            value="",
            description="Các tác vụ:",
            placeholder="e.g. 1, 17, 2, 3",
            style={"description_width": "initial", "font_size": "20px"},
            layout=Layout(width="45%"),
        )
        for _ in range(num_jobs.value)
    ]
    orders_input = [
        Text(
            value="",
            description="Thứ tự thực hiện:",
            placeholder="e.g. (1, 17), (17, 2), (2, 3)",
            style={"description_width": "initial", "font_size": "20px"},
            layout=Layout(width="45%"),
        )
        for _ in range(num_jobs.value)
    ]
    times = [
        widgets.IntText(min=1, max=1000, value=1, description="Số lần: ")
        for _ in range(num_jobs.value)
    ]
    # Step 6
    num = num_jobs.value
    for i, (task, order, time) in enumerate(zip(tasks_input, orders_input, times)):
        task.add_class("mytext")
        order.add_class("mytext")
        time.add_class("mytext")
        tasks_orders = VBox(
            [
                Label(
                    "Công việc %d:" % (i + 1),
                    style={"font_size": "20px", "text_color": "#0fa312"},
                ),
                HBox(
                    [task, order, time],
                    layout=widgets.Layout(
                        display="flex",
                        justify_content="space-around",
                    ),
                ),
            ]
        )
        # Step 9
        with output:
            display(tasks_orders)


# Step 8
button = widgets.Button(
    description="Nhấn để nhập thông tin về các công việc",
    style={"font_weight": "bold", "font_size": "15px", "button_color": "pink"},
    layout=widgets.Layout(width="25%", height="50px"),
)
button.add_class("mytext")
button.on_click(on_button_click)
solve_button = Button(
    description="Solve",
    button_style="success",
    tooltip="Click me",
    icon="check",
    style={"font_weight": "bold", "font_size": "15px"},
)
model_selector = widgets.RadioButtons(
    options=["MIP", "Greedy"],
    value="MIP",
    layout={"width": "max-content", "font_size": "20px"},
    description="Chọn mô hình:",
    disabled=False,
)
display(num_jobs, button, output, model_selector, solve_button)

tasks = []
orders = []
model = None


def bind_solve_button(b):
    # clear all errors output
    global tasks, orders, tasks_input, orders_input, times
    tasks = []
    orders = []
    for task in tasks_input:
        if task.value == "":
            raise ValueError("Các tác vụ không được để trống")
        t = eval(task.value)
        if type(t) is int:
            tasks.append([t])
        else:
            tasks.append(t)
    for order in orders_input:
        if order.value == "":
            orders.append([])
            continue
        l = eval(order.value)
        if type(l[0]) is int:
            orders.append([l])
        else:
            orders.append(l)
    assert all(
        [not checkCycle(order) for order in orders]
    ), "Tồn tại chu trình trong các thứ tự thực hiện"
    df = pd.read_excel("../datasets/Data.xlsx", sheet_name="Tasks")
    all_tasks = set(df["Task ID"])
    for task in tasks:
        assert all(
            [t in all_tasks for t in task]
        ), "Có tác vụ không tồn tại trong bảng Tasks"
    for i, order in enumerate(orders):
        tasks_in_job = set().union(*[set(i) for i in order])
        assert all(
            [t in tasks[i] for t in tasks_in_job]
        ), "Có tác vụ không tồn tại trong danh sách tác vụ của công việc"
    repeats = [i.value for i in times]
    assert all(i > 0 for i in repeats), "Số lần thực hiện phải lớn hơn 0"
    global model, model_selector
    if model_selector.value == "MIP":
        model = LPJS(tasks, orders, capacities, repeats)
    else:
        model = GreedyJS(tasks, orders, capacities, repeats)
    model.solve()


solve_button.on_click(bind_solve_button)

IntText(value=1, description='Chọn số lượng công việc:', layout=Layout(width='25%'), style=DescriptionStyle(de…

Button(description='Nhấn để nhập thông tin về các công việc', layout=Layout(height='50px', width='25%'), style…

Output()

RadioButtons(description='Chọn mô hình:', layout=Layout(width='max-content'), options=('MIP', 'Greedy'), value…

Button(button_style='success', description='Solve', icon='check', style=ButtonStyle(font_size='15px', font_wei…